In [1]:
%pwd

'C:\\NN-P4\\python-notebooks'

###Imports and Read CSV

In [1]:
import json
import numpy as np
import pandas as pd
import tensorflow as tf
import os
from datetime import datetime
from keras.callbacks import History # to get train accuracy
history = History()

In [2]:
#nPrintML creates a copy of dataset in pkl picles format. Convert to CSV 1st.
# Read the PKL
df_train = pd.read_pickle('../datasets/nprint-raw/stream-video-half-c5-f-r/X.pkl')
df_test = pd.read_pickle('../datasets/nprint-raw/stream-video-half-c5-f-r/X_val.pkl')
label_train = pd.read_pickle('../datasets/nprint-raw/stream-video-half-c5-f-r/y.pkl')
label_test = pd.read_pickle('../datasets/nprint-raw/stream-video-half-c5-f-r/y_val.pkl')
# Save a copy as CSV
df_train.to_csv('../datasets/nprint-raw/stream-video-half-c5-f-r/X.csv')
df_test.to_csv('../datasets/nprint-raw/stream-video-half-c5-f-r/X_val.csv')
label_train.to_csv('../datasets/nprint-raw/stream-video-half-c5-f-r/y.csv')
label_test.to_csv('../datasets/nprint-raw/stream-video-half-c5-f-r/y_val.csv')
# Read CSV
df_train = pd.read_csv('../datasets/nprint-raw/stream-video-half-c5-f-r/X.csv')
df_test = pd.read_csv('../datasets/nprint-raw/stream-video-half-c5-f-r/X_val.csv')
label_train = pd.read_csv('../datasets/nprint-raw/stream-video-half-c5-f-r/y.csv')
label_test = pd.read_csv('../datasets/nprint-raw/stream-video-half-c5-f-r/y_val.csv')
#Drop colum with pcap sequence number
df_train.drop('pcap', axis=1, inplace=True)
df_test.drop('pcap', axis=1, inplace=True)
label_train.drop('pcap', axis=1, inplace=True)
label_test.drop('pcap', axis=1, inplace=True)
#Rename Unnamed Collumn
label_train.rename(columns={'0': 'label'}, inplace=True)
label_test.rename(columns={'0': 'label'}, inplace=True)

FileNotFoundError: [Errno 2] No such file or directory: '../datasets/nprint-raw/stream-video-half-c5-f-r/X.pkl'

In [3]:
display(df_train)
display(df_test)
display(label_train)
display(label_test)
print(label_train["label"].value_counts())
print(label_test["label"].value_counts())
print(f"df_train shape: {df_train.shape} \n columns: {df_train.columns.values}")
print(f"df_test shape: {df_test.shape} \n columns: {df_test.columns.values}")

NameError: name 'df_train' is not defined

In [49]:
#Read file with ordered list of attributes ranked by the autoML feature importance algorithm
features_rankings_df = pd.read_csv('../datasets/nprint-raw/stream-video-c10-little/feature_importance-c10.csv')
features_rankings_df.rename(columns={'0': 'label'}, inplace=True)

feature_list = features_rankings_df['Unnamed: 0'].tolist()
feature_list_forbiden = ['pkt_7_ipv4_dst_13','pkt_1_ipv4_dst_16', 'pkt_8_ipv4_src_13', 'pkt_2_ipv4_dst_25', 'pkt_4_ipv4_dst_6', 'pkt_8_ipv4_src_16', 'pkt_5_ipv4_dst_1','pkt_2_ipv4_dst_9', 'pkt_6_ipv4_dst_5', 'pkt_4_ipv4_dst_7', 'pkt_9_ipv4_dst_18', 'pkt_7_ipv4_dst_6', 'pkt_6_ipv4_dst_6', 'pkt_9_ipv4_src_13', 'pkt_8_ipv4_src_17','pkt_9_ipv4_dst_6', 'pkt_9_ipv4_dst_13', 'pkt_5_ipv4_dst_6', 'pkt_2_ipv4_dst_13', 'pkt_9_ipv4_dst_16', 'pkt_9_ipv4_dst_0', 'pkt_9_ipv4_dst_1', 'pkt_7_ipv4_dst_0', 'pkt_4_ipv4_dst_0', 'pkt_8_ipv4_dst_7', 'pkt_3_ipv4_dst_1', 'pkt_2_ipv4_dst_7', 'pkt_7_ipv4_src_3']

l3 = [x for x in feature_list if x not in feature_list_forbiden]

#print(l3)

l3 = l3[:32]

print(l3)

['pkt_2_rts', 'pkt_6_rts', 'pkt_4_rts', 'pkt_5_rts', 'pkt_1_rts', 'pkt_8_rts', 'pkt_9_rts', 'pkt_7_rts', 'pkt_3_rts', 'pkt_8_ipv4_ttl_5', 'pkt_3_tcp_opt_159', 'pkt_1_tcp_wsize_4', 'pkt_0_tcp_opt_64', 'pkt_3_tcp_wsize_4', 'pkt_1_tcp_opt_159', 'pkt_8_ipv4_ttl_1', 'pkt_1_tcp_opt_96', 'pkt_6_ipv4_id_6', 'pkt_1_tcp_seq_8', 'pkt_9_tcp_wsize_0', 'pkt_1_tcp_wsize_1', 'pkt_6_tcp_seq_18', 'pkt_7_tcp_wsize_5', 'pkt_8_tcp_seq_8', 'pkt_5_tcp_dprt_5', 'pkt_8_tcp_opt_37', 'pkt_0_tcp_seq_24', 'pkt_5_tcp_seq_19', 'pkt_9_tcp_ackn_26', 'pkt_2_tcp_sprt_5', 'pkt_8_tcp_opt_29', 'pkt_8_tcp_ackn_10']


In [50]:
#select a sub-dataframe with only the 32 top attrs rate by the AUTOGLUON feature importance algorithm
display(df_train)
display(df_test)
df_train = df_train[l3]
df_test = df_test[l3]
display(df_train)
display(df_test)

,pkt_0_ipv4_tl_9,pkt_0_ipv4_tl_10,pkt_0_ipv4_tl_11,pkt_0_ipv4_tl_12,pkt_0_ipv4_tl_13,pkt_0_ipv4_id_0,pkt_0_ipv4_id_1,pkt_0_ipv4_id_2,pkt_0_ipv4_id_3,pkt_0_ipv4_id_4,...,pkt_9_tcp_opt_182,pkt_9_tcp_opt_183,pkt_9_tcp_opt_184,pkt_9_tcp_opt_185,pkt_9_tcp_opt_186,pkt_9_tcp_opt_187,pkt_9_tcp_opt_188,pkt_9_tcp_opt_189,pkt_9_tcp_opt_190,pkt_9_tcp_opt_191
0,0,1,1,1,1,1,0,0,0,1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,0,1,1,0,1,0,1,0,0,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,0,1,1,1,1,0,0,1,1,1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3,1,0,0,0,0,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6592,0,1,1,1,1,1,1,0,0,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
6593,1,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
6594,0,1,1,1,1,0,0,0,1,1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
6595,0,1,1,1,1,0,1,0,1,1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


,pkt_0_ipv4_tl_9,pkt_0_ipv4_tl_10,pkt_0_ipv4_tl_11,pkt_0_ipv4_tl_12,pkt_0_ipv4_tl_13,pkt_0_ipv4_id_0,pkt_0_ipv4_id_1,pkt_0_ipv4_id_2,pkt_0_ipv4_id_3,pkt_0_ipv4_id_4,...,pkt_9_tcp_opt_182,pkt_9_tcp_opt_183,pkt_9_tcp_opt_184,pkt_9_tcp_opt_185,pkt_9_tcp_opt_186,pkt_9_tcp_opt_187,pkt_9_tcp_opt_188,pkt_9_tcp_opt_189,pkt_9_tcp_opt_190,pkt_9_tcp_opt_191
0,0,1,1,1,1,0,0,0,1,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,0,1,1,1,1,0,0,1,1,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,0,1,1,1,1,1,1,0,1,1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3,0,1,1,1,1,0,1,0,0,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
4,1,0,0,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
728,1,0,0,0,0,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
729,1,0,0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
730,0,1,1,1,1,0,0,1,0,1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
731,1,0,0,0,0,1,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0


,pkt_2_rts,pkt_6_rts,pkt_4_rts,pkt_5_rts,pkt_1_rts,pkt_8_rts,pkt_9_rts,pkt_7_rts,pkt_3_rts,pkt_8_ipv4_ttl_5,...,pkt_7_tcp_wsize_5,pkt_8_tcp_seq_8,pkt_5_tcp_dprt_5,pkt_8_tcp_opt_37,pkt_0_tcp_seq_24,pkt_5_tcp_seq_19,pkt_9_tcp_ackn_26,pkt_2_tcp_sprt_5,pkt_8_tcp_opt_29,pkt_8_tcp_ackn_10
0,13470,23,17451,3397,44,20,20,20,35,0,...,0,1,0,1,1,1,0,0,1,0
1,1126,66559,2377,246468,1327,490,3139,4094,1753,0,...,0,1,0,0,0,0,0,0,1,0
2,1237564,9622,59,579,381,123,59,67,68,0,...,0,1,0,1,1,0,0,1,1,0
3,1514911,327280,587699,1097406,223,245,2055571,649932,609443,0,...,1,0,0,0,0,0,0,1,1,0
4,675961,355,103,440463,19637,92,134,3164,1483863,0,...,1,1,0,0,0,1,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6592,1307587,58,144,56,532,65,58,249,4826,0,...,0,1,0,1,0,0,0,1,1,0
6593,168,97,184,251552,166,72,78,51,138,0,...,1,1,0,0,1,0,0,0,1,0
6594,1264139,65,88,109634,384,54,389,56,111,0,...,0,1,0,1,0,0,0,1,1,0
6595,37,54,31,81,229139,37,29,2704,33,0,...,0,1,0,1,1,0,0,1,1,0


,pkt_2_rts,pkt_6_rts,pkt_4_rts,pkt_5_rts,pkt_1_rts,pkt_8_rts,pkt_9_rts,pkt_7_rts,pkt_3_rts,pkt_8_ipv4_ttl_5,...,pkt_7_tcp_wsize_5,pkt_8_tcp_seq_8,pkt_5_tcp_dprt_5,pkt_8_tcp_opt_37,pkt_0_tcp_seq_24,pkt_5_tcp_seq_19,pkt_9_tcp_ackn_26,pkt_2_tcp_sprt_5,pkt_8_tcp_opt_29,pkt_8_tcp_ackn_10
0,1282712,57,849,58,295,111,55,480,89,0,...,0,1,0,1,0,1,0,1,1,0
1,1379684,420,144,91,253,57,57,73,4824,0,...,0,0,0,1,0,0,0,1,1,0
2,930898,99,68,60,300,591,59,58,284384,0,...,0,1,0,1,1,1,0,0,1,0
3,877,77,166,96,1237315,74,65,69,177,0,...,0,0,0,1,0,0,0,0,1,0
4,1647622,57,49,3349,206,1102,51,49,61,0,...,1,1,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
728,54,51,4257,50,1375651,74,53,48,55,0,...,1,1,0,0,0,0,0,0,1,0
729,172,887,1270836,8395,12689,89,48,1415,272,0,...,1,1,0,0,1,0,0,1,1,0
730,892114,10371,201499835,130200,400646,361189895,51786,156184566,910096,0,...,0,1,0,1,0,0,0,0,1,0
731,531841,77,355,905661,1486,74,73,130,74988,0,...,1,1,0,0,1,0,0,0,1,0


In [51]:
# Model Parameters
number_of_attributes = df_train.shape[1]
hidden_layer_nodes = 32
output_layer_nodes = 4

# Traing Parameters
EPOCHS = 500
BATCH_SIZE = 500

# Run the NN Training multiple times to get a nice mean score of accuracy results
NUMBER_OF_NN_REPETITIONS = 10

def get_basic_model():
  model = tf.keras.Sequential([
    normalizer,
    tf.keras.layers.Dense(hidden_layer_nodes, activation='relu'),
    tf.keras.layers.Dense(output_layer_nodes)
  ])
  model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
  return model

In [52]:
# Convert int values to TF expect values (float)
df_train = np.asarray(df_train).astype(np.float32)
df_test = np.asarray(df_test).astype(np.float32)
# Converts pandas dataframe to tensorflow object
df_train = tf.convert_to_tensor(df_train)
# Set the 
normalizer = tf.keras.layers.Normalization(axis=-1)
# Normalize the data
normalizer.adapt(df_train)
#normalizer.adapt(numeric_features_test)

In [53]:
# Execute Trainning and Testing
#model = get_basic_model()
train_accuracy = []
test_accuracy = []

for i in range(NUMBER_OF_NN_REPETITIONS):
  model = get_basic_model()
  history = model.fit(df_train, label_train, epochs=EPOCHS, verbose=0, batch_size=BATCH_SIZE, callbacks=[history])
  one_train_accuracy = history.history['accuracy']

  test_loss, one_test_accuracy =  model.evaluate(df_test,  label_test, verbose=1, batch_size=BATCH_SIZE)
  train_accuracy.append(one_train_accuracy[-1])
  test_accuracy.append(one_test_accuracy)
  print(f"{i+1}: train_accuracy:{train_accuracy} test_accuracy:{test_accuracy}")
  tf_predictions_probabilities = model.predict(df_test)
  #tf_loss, tf_acc =  model.evaluate(numeric_features_test,  label_test, verbose=2, batch_size=BATCH_SIZE)

  #Confusion Matrix
  tf_predictions = []
  for i,x in enumerate(tf_predictions_probabilities):
    #print("i:",i,"x_max:",x.max(),"x:",x)
    j_max = x.argmax()
    tf_predictions.append(j_max)
  conf_m = tf.math.confusion_matrix(label_test,tf_predictions)
  print(conf_m)

#After n repetitions of training and testing, calculate statistics
train_accuracy_mean = sum(train_accuracy) / len(train_accuracy)
train_max = max(train_accuracy)
train_min = min(train_accuracy)
train_std = np.std(train_accuracy)
test_accuracy_mean = sum(test_accuracy) / len(test_accuracy)
test_max = max(test_accuracy)
test_min = min(test_accuracy)
test_std = np.std(test_accuracy)

print(f'accuracies_train: {train_accuracy}, accuracies_test: {test_accuracy}')
print(f'train_accuracy_mean: {train_accuracy_mean*100:.2f}, train_max: {train_max*100:.2f}, train_min: {train_min*100:.2f}, train_std: {train_std*100:.2f} \n test_accuracy_mean: {test_accuracy_mean*100:.2f}, test_max: {test_max*100:.2f}, test_min: {test_min*100:.2f}, test_std: {test_std*100:.2f}')



2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5467 - loss: 0.8717 
1: train_accuracy:[0.5889040231704712] test_accuracy:[0.547066867351532]
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
tf.Tensor(
[[ 20   5  23  19]
 [  6 113  29  73]
 [ 18  42  34  20]
 [ 11  72  14 234]], shape=(4, 4), dtype=int32)
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5706 - loss: 0.8710 
2: train_accuracy:[0.5889040231704712, 0.5858723521232605] test_accuracy:[0.547066867351532, 0.5688949227333069]
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
tf.Tensor(
[[ 17   7  25  18]
 [  3 125  29  64]
 [ 10  43  41  20]
 [  8  79  10 234]], shape=(4, 4), dtype=int32)
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5605 - loss: 0.8667 
3: train_accuracy:[0.5889040231704712, 0.5858723521232605, 0.5810216665267944] test_accuracy:[0.547066867351532, 0.5688949227333069, 0.5607094168663025]
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
tf.Tensor(
[[ 18  10  22  17]
 [  5 120  25  71]
 [ 12  50  29  23]
 [  6  73   8 244]], shap

In [12]:
# Date for report file
now = datetime.now()
dt_string = now.strftime("%d-%m-%Y-%H-%M-%S")
print(dt_string)
# Report file content
report = {
    "datetime": dt_string,
    "training_dataset_shape":  str(df_train.shape),
    "test_dataset_shape": str(df_test.shape),
    "number_of_attributes": number_of_attributes,
    "n_epoch":EPOCHS,
    "accuracy_train": train_accuracy_mean,
    "train_max": train_max,
    "train_min": train_min,
    "train_std": train_std,
    "accuracy_test": test_accuracy_mean,
    "test_max": test_max,
    "test_min": test_min,
    "test_std": test_std,
    "accuracies_train": train_accuracy,
    "accuracies_test": test_accuracy
}

# Writes into the file
os.chdir('../tf-model-testing')
print(os.getcwd())

title_parameters_save = f"nn-nprint-app-iden-model-evaluation-{number_of_attributes}x{hidden_layer_nodes}x{output_layer_nodes}.json"
with open(title_parameters_save, "w") as f:
  json.dump(report, f)

title_parameters_save = f"nn-nprint-app-iden-model-evaluation-{number_of_attributes}x{hidden_layer_nodes}x{output_layer_nodes}-{dt_string}.json"
with open(title_parameters_save, "w") as f:
  json.dump(report, f)

12-06-2024-16-27-20
C:\NN-P4\tf-model-testing
